### **Loading data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape, Dropout
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from keras.models import Sequential, Model
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import cv2
import os
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
np.random.seed(11)

In [ ]:
train_dir = os.path.join(os. getcwd() , r'/content/drive/MyDrive/train')
test_dir = os.path.join (os.getcwd(), r'/content/drive/MyDrive/test')
categories = ['benign','malignant']
y_train = []
x_train = []
x_test = []
y_test = []
Batch_Size = 32
INIT_LR = 1e-4
EPOCHES = 10

In [ ]:
for category in categories:
  path = os.path.join(train_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image = cv2.resize(image, (224,224))
    image = image / 255
    x_train.append(image)
    y_train.append(category)

In [ ]:
for category in categories:
  path = os.path.join(test_dir, category)
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path)
    image - cv2.resize(image, (224,224))
    image = image / 255
    x_test.append(image)
    y_test.append(category)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical

lb = LabelBinarizer ()
y_train = lb.fit_transform(y_train)
y_train = to_categorical(y_train,num_classes=2)
y_test = lb.fit_transform(y_test)
y_test = to_categorical (y_test, num_classes=2)

In [ ]:
x_train = np.array(x_train, dtype= 'float32')
y_train = np.array(y_train,dtype= 'float32')
x_test = np.array(x_test,dtype= 'float32')
y_test = np.array(y_test, dtype= 'float32')

In [ ]:
x_train.shape

(2637, 224, 224, 3)

In [ ]:
x_test.shape

(660, 224, 224, 3)

In [ ]:
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:]))) #Flattn the 224*224*3 images into vectors of size 150528
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

x_train = x_train.reshape(-1, 150528)  # Reshape to (number_of_samples, 150528)
x_test = x_test.reshape(-1, 150528)  # Reshape to (number_of_samples, 150528)

In [ ]:
x_train.shape

(2637, 150528)

### **"2"**

In [ ]:
input_dim = 150528
input_shape = (input_dim,)

# Encoder
encoder_input = Input(shape=input_shape)
encoded = Dense(128, activation='relu') (encoder_input)
encoded = Dense(64, activation= 'relu') (encoded)

# Decoder
decoded = Dense(64, activation= 'relu') (encoded)
decoded = Dense(128, activation='relu') (decoded)
decoded = Dense(input_dim, activation='sigmoid') (decoded) # Last Decoder Output

In [ ]:
autoencoder = Model(encoder_input, decoded)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(x_train, x_train, epochs=EPOCHES, batch_size=Batch_Size)

Epoch 1/10
83/83 [==============================] - 59s 684ms/step - loss: 0.0206
Epoch 2/10
83/83 [==============================] - 64s 768ms/step - loss: 0.0142
Epoch 3/10
83/83 [==============================] - 56s 678ms/step - loss: 0.0127
Epoch 4/10
83/83 [==============================] - 57s 690ms/step - loss: 0.0121
Epoch 5/10
83/83 [==============================] - 58s 701ms/step - loss: 0.0116
Epoch 6/10
83/83 [==============================] - 57s 684ms/step - loss: 0.0108
Epoch 7/10
83/83 [==============================] - 59s 714ms/step - loss: 0.0104
Epoch 8/10
83/83 [==============================] - 63s 758ms/step - loss: 0.0100
Epoch 9/10
83/83 [==============================] - 60s 728ms/step - loss: 0.0098
Epoch 10/10
83/83 [==============================] - 58s 703ms/step - loss: 0.0094


In [ ]:
# Classification Layer
classification_output = Dense(2, activation='softmax') (encoded)

# Create the classification model
classifier_model = Model(encoder_input, classification_output)

# Compile the classification model
classifier_model.compile(
    optimizer=Adam(learning_rate=INIT_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# Train the classifier
classifier_model.fit(x_train, y_train, epochs=EPOCHES, batch_size=Batch_Size)

# Evaluate the classifier
test_loss, test_accuracy = classifier_model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/10
83/83 [==============================] - 28s 330ms/step - loss: 1.0717 - accuracy: 0.6140
Epoch 2/10
83/83 [==============================] - 30s 363ms/step - loss: 0.7917 - accuracy: 0.6542
Epoch 3/10
83/83 [==============================] - 30s 357ms/step - loss: 0.6853 - accuracy: 0.7103
Epoch 4/10
83/83 [==============================] - 32s 389ms/step - loss: 0.8300 - accuracy: 0.6780
Epoch 5/10
83/83 [==============================] - 29s 345ms/step - loss: 0.5833 - accuracy: 0.7410
Epoch 6/10
83/83 [==============================] - 29s 351ms/step - loss: 0.6535 - accuracy: 0.7148
Epoch 7/10
83/83 [==============================] - 29s 346ms/step - loss: 0.5537 - accuracy: 0.7467
Epoch 8/10
83/83 [==============================] - 28s 333ms/step - loss: 0.5882 - accuracy: 0.7364
Epoch 9/10
83/83 [==============================] - 27s 332ms/step - loss: 0.5625 - accuracy: 0.7535
Epoch 10/10
21/21 [==============================] - 1s 40ms/step - loss: 0.5312 - accuracy

### **"3"**

**Reshaping output vector of the last decoder layer**

In [ ]:
decoded_reshaped = Reshape((224, 224, 3))(decoded)

**CNN Layers**

In [ ]:
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(decoded_reshaped)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(max_pool2)

**Classification**

In [ ]:
# Add Dense layers for classification
dense1 = Dense(128, activation='relu')(flatten)
dropout = Dropout(0.5)(dense1)
classification_output = Dense(2, activation='softmax')(dropout)

# Create the classification model
classifier = Model(encoder_input, classification_output)

# Compile the classification model
classifier.compile(
    optimizer=Adam(learning_rate=INIT_LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# Train the classifier (replace with your actual training data)
classifier.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
83/83 [==============================] - 294s 3s/step - loss: 0.7128 - accuracy: 0.5256
Epoch 2/10
83/83 [==============================] - 298s 4s/step - loss: 0.6604 - accuracy: 0.5506
Epoch 3/10
83/83 [==============================] - 289s 3s/step - loss: 0.6066 - accuracy: 0.6261
Epoch 4/10
83/83 [==============================] - 297s 4s/step - loss: 0.5616 - accuracy: 0.7133
Epoch 5/10
83/83 [==============================] - 297s 4s/step - loss: 0.5875 - accuracy: 0.6868
Epoch 6/10
83/83 [==============================] - 293s 4s/step - loss: 0.5367 - accuracy: 0.7444
Epoch 7/10
83/83 [==============================] - 314s 4s/step - loss: 0.5020 - accuracy: 0.7550
Epoch 8/10
83/83 [==============================] - 299s 4s/step - loss: 0.4652 - accuracy: 0.7979
Epoch 9/10
83/83 [==============================] - 295s 4s/step - loss: 0.4870 - accuracy: 0.7823
Epoch 10/10
83/83 [==============================] - 293s 4s/step - loss: 0.4826 - accuracy: 0.7778


In [ ]:
test_loss, test_accuracy = classifier.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

21/21 [==============================] - 19s 887ms/step - loss: 0.4230 - accuracy: 0.7924
Test Loss: 0.4230
Test Accuracy: 79.24%


### **"4"**

In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

**Reshaping output vector of the last decoder layer**

In [ ]:
decoded_reshaped = Reshape((224, 224, 3))(decoded)

In [ ]:
# Create the autoencoder model
autoencoder = Model(encoder_input, decoded_reshaped)

**ResNet50**

In [ ]:
# Input layer for ResNet50
input_resnet = Input(shape=(224, 224, 3))

# Load the pre-trained ResNet50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_resnet)

**CNN**

In [ ]:
x = Conv2D(32, kernel_size=(3, 3), activation='relu')(decoded_reshaped)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)

**Classification**

In [ ]:
# classification layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
classification_output = Dense(2, activation='softmax')(x)

In [ ]:
# Connect ResNet50 to the classifier
resnet_output = base_model(input_resnet)
x = Flatten()(resnet_output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
classification_output = Dense(2, activation='softmax')(x)

In [ ]:
# Create the classification model
classifier = Model(inputs=[encoder_input, input_resnet], outputs=classification_output)

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
Reshape((224, 224, 3))(x_train).shape

TensorShape([2637, 224, 224, 3])

In [ ]:
classifier.fit([x_train, Reshape((224, 224, 3))(x_train)],
               y_train,
               epochs=2,
               batch_size=20)

Epoch 1/2
132/132 [==============================] - 2197s 16s/step - loss: 1.8984 - accuracy: 0.7478
Epoch 2/2
132/132 [==============================] - 2177s 16s/step - loss: 0.4605 - accuracy: 0.7835
